In [1]:
from neuromeka import EtherCAT
import time
import math

In [2]:
ip = '160.39.102.105'
ecat = EtherCAT(ip)

In [235]:
servos = [
    {"index": 0, "direction": -1, "gear_ratio": 121, "ppr": 65536, "max_ecat_torque": 48.0, "rated_torque": 0.08839, "version":  "", "correction_rad": -0.054279739737023644},
    {"index": 1, "direction": -1, "gear_ratio": 121, "ppr": 65536, "max_ecat_torque": 48.0, "rated_torque": 0.0839705, "version":  "", "correction_rad": -0.013264502315156903},
    {"index": 2, "direction": 1, "gear_ratio": 121, "ppr": 65536, "max_ecat_torque": 96.0, "rated_torque": 0.0891443, "version":  "", "correction_rad": 2.794970264143719},
    {"index": 3, "direction": -1, "gear_ratio": 101, "ppr": 65536, "max_ecat_torque": 96.0, "rated_torque": 0.05798, "version":  "", "correction_rad": -0.0054105206811824215},
    {"index": 4, "direction": -1, "gear_ratio": 101, "ppr": 65536, "max_ecat_torque": 96.0, "rated_torque": 0.055081, "version":  "", "correction_rad": 2.7930504019665254},
    {"index": 5, "direction": -1, "gear_ratio": 101, "ppr": 65536, "max_ecat_torque": 96.0, "rated_torque": 0.05798, "version":  "", "correction_rad": -0.03490658503988659}
]
min_torques = [240, 600, 250, 180, 140, 40] # empirical min torques to move each joint at max load
min_pos_deg = [-175.0, -175.0, -175.0, -175.0, -175.0, -215.0]
max_pos_deg = [ 175.0, 175.0, 175.0, 175.0, 175.0, 215.0]
min_pos_rad = [min_pos_deg[i] * math.pi / 180 for i in range(6)]
max_pos_rad = [max_pos_deg[i] * math.pi / 180 for i in range(6)]

In [4]:
print("Master status: ", ecat.get_master_status())
print("Slave status: ", ecat.get_slave_status())
print("TxPDO domain: ", ecat.get_txdomain_status())
print("RxPDO domain: ", ecat.get_rxdomain_status())
print("System ready: ", ecat.is_system_ready())
print("Servo state: ", ecat.is_servo_on())
print("")
print(ecat.get_slave_type_num())
num_servo = ecat.get_slave_type_num().num_servo

Master status:  OP
Slave status:  ['OP', 'OP', 'OP', 'OP', 'OP', 'OP', 'OP', 'OP']
TxPDO domain:  COMPLETE
RxPDO domain:  COMPLETE
System ready:  [False, False, False, False, False, False]
Servo state:  [False, False, False, False, False, False]

num_servo: 6
num_ioboard: 1
num_endtool: 1



In [256]:
# ENABLE ALL SERVOS
# ecat.set_servo(0, True)
# ecat.set_servo(1, True)
# ecat.set_servo(2, True)
# ecat.set_servo(3, True)
ecat.set_servo(4, True)
# ecat.set_servo(5, True)


In [257]:
# DISABLE ALL SERVOS
ecat.set_servo(0, False)
ecat.set_servo(1, False)
ecat.set_servo(2, False)
ecat.set_servo(3, False)
ecat.set_servo(4, False)
ecat.set_servo(5, False)

In [7]:
# GET STATE OF ALL SERVOS
for i in range(0, 6):
    print(ecat.get_servo_tx(i))

['READY_SWITCH', 'CST', 179747, 0, 1]
['READY_SWITCH', 'CST', 170819, 0, -20]
['READY_SWITCH', 'CST', -5891173, 0, 96]
['READY_SWITCH', 'CST', -125075, 0, 56]
['READY_SWITCH', 'CST', -2505216, 0, -107]
['READY_SWITCH', 'CST', 247016, 0, -1]


In [140]:
# GET POSITIONS IN DEGREES
for i in range(6):
    ppr = servos[i]["ppr"]
    gear_ratio = servos[i]["gear_ratio"]
    pos, vel, tor = ecat.get_servo_tx(i)[2:5]
    pos_rad = ((2 * math.pi * pos / gear_ratio / ppr) + servos[i]["correction_rad"]) * servos[i]["direction"]
    vel_rad = 2 * math.pi *vel / gear_ratio / ppr * servos[i]["direction"]
    pos_deg = pos_rad * 180 / math.pi
    print(f'{round(pos_rad, 2)} rad\t{round(pos_deg, 2)} deg\t{round(vel_rad, 2)} rad/s')

0.18 rad	10.07 deg	-0.0 rad/s
0.39 rad	22.5 deg	-0.0 rad/s
-0.07 rad	-3.78 deg	0.0 rad/s
0.54 rad	31.06 deg	-0.0 rad/s
-0.14 rad	-7.97 deg	-0.0 rad/s
0.0 rad	0.02 deg	-0.0 rad/s


In [10]:
# MOVE SINGLE SERVO TO POSITION
def goto(servo_idx, goal_position):
    ppr = servos[servo_idx]["ppr"]
    gear_ratio = servos[servo_idx]["gear_ratio"]
    targetTorque = min_torques[servo_idx]
    positive_start_pos = None

    ecat.set_servo(servo_idx, True)
    while 1:
        pos, vel = ecat.get_servo_tx(servo_idx)[2:4]
        pos_rad = ((2 * math.pi * pos / gear_ratio / ppr) + servos[servo_idx]["correction_rad"]) * servos[servo_idx]["direction"]

        # break on sign change
        if positive_start_pos is None:
            positive_start_pos = pos_rad > goal_position
        if positive_start_pos and pos_rad < goal_position or not positive_start_pos and pos_rad > goal_position:
            break

        # break on goal position
        if abs(pos_rad - goal_position) < 0.001:
            break

        # break on min/max position
        if pos_rad < min_pos_rad[servo_idx] + 0.01 or pos_rad > max_pos_rad[servo_idx] - 0.01:
            break

        # move to goal position
        direction = servos[servo_idx]["direction"] if goal_position > pos_rad else -servos[servo_idx]["direction"]
        ecat.set_servo_rx(servo_idx, 0x0f, 0x0a, 0, 0, targetTorque*direction)

    # stop servo
    ecat.set_servo_rx(servo_idx, 0x0f, 0x0a, 0, 0, 0)
    ecat.set_servo(servo_idx, False)

In [282]:
# DISABLE ALL SERVOS
ecat.set_servo(0, False) 
ecat.set_servo(1, False)
ecat.set_servo(2, False)
ecat.set_servo(3, False)    
ecat.set_servo(4, False)
ecat.set_servo(5, False) 

In [283]:
# ZERO TORQUE ALL SERVOS
for i in range(6):
    ecat.set_servo(i, True)
    ecat.set_servo_rx(i, 0x0f, 0x0a, 0, 0, 0)
    ecat.set_servo(i, False)

In [284]:
# Next 5 cells: start position

In [285]:
goto(1, 0.3)

In [286]:
goto(0,0)

In [287]:
# goto(2, - 3 * math.pi / 4)
goto(2, -(math.pi / 2) - 0.3)

In [288]:
goto(3, 0)

In [289]:
# goto(4,1.3)
goto(4, 0)

In [252]:
goto(5, 0)

In [11]:
# OTHER STUFF

In [162]:
print("Temperature")    
for i in range(0,6):
    print(i, ecat.get_servo_temperature(i))
    
print("Error code")    
for i in range(0,6):    
    print(i, hex(ecat.get_servo_errorcode(i)))

Temperature
0 36.2548828125
1 37.060546875
2 40.36376953125
3 41.4111328125
4 43.3447265625
5 43.74755859375
Error code
0 0x0
1 0x0
2 0x0
3 0x0
4 0x0
5 0x0


In [13]:
ecat.set_brake(1, False)
time.sleep(1)
ecat.set_brake(1, True)

In [14]:
print(ecat.get_endtool_tx())
ecat.get_endtool_rx()

{'status': 0, 'button': 0, 'ft_sensor': , 'ft_state': 0, 'ft_error': 0}


{'eqc': 0,
 'gripper': 0,
 'ft_param': 0,
 'led_mode': 0,
 'led_g': 1,
 'led_r': 1,
 'led_b': 1}

In [15]:
ecat.set_endtool_rx({"eqc": 0, "gripper": 0, "ft_param": 0, "led_mode": 0, "led_g": 1, "led_r": 1, "led_b": 1})